In [25]:
import os
from pathlib import Path
from tqdm import tqdm#_notebook as tqdm
from pku_autonomous_driving import io, util, dataset, resnet, centernet, prediction, graphics

import importlib
importlib.reload(io)
importlib.reload(util)
importlib.reload(dataset)
importlib.reload(resnet)
importlib.reload(centernet)
importlib.reload(prediction)
importlib.reload(graphics)

<module 'pku_autonomous_driving.graphics' from '/Users/argon/workspace/managed/pku-autonomous-driving/pku_autonomous_driving/graphics.py'>

In [26]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [27]:
import torchvision
from pku_autonomous_driving.transform import CropBottomHalf, PadByMean, Resize, Normalize, DropPointsAtOutOfScreen, CreateMask, CreateRegr, ToCHWOrder
from pku_autonomous_driving.const import IMG_WIDTH, IMG_HEIGHT, MODEL_SCALE

test_transform = torchvision.transforms.Compose([
    CropBottomHalf(),
    PadByMean(),
    Resize(IMG_WIDTH, IMG_HEIGHT),
    Normalize(),
    ToCHWOrder()
])

In [30]:
from pku_autonomous_driving.dataset import CarDataset, create_data_loader

test = io.load_test_data()

test_dataset = CarDataset(test, training=False, transform=test_transform)
test_loader = create_data_loader(test_dataset, batch_size=1)

In [31]:
base_model = resnet.resnext50_32x4d(pretrained=False)
model = centernet.CentResnet(base_model, 8)

setup_kwargs = {"model": model, "device": device}
if 'INITIAL_WEIGHTS' in os.environ:
    setup_kwargs["path"] = Path(os.environ["INITIAL_WEIGHTS"])
util.setup_model(**setup_kwargs)

In [32]:
%%time
predictions = prediction.predict(model, test_loader, device)

CPU times: user 44.6 s, sys: 4.28 s, total: 48.9 s
Wall time: 50.1 s


In [39]:
test = io.load_test_data_raw()
test['PredictionString'] = predictions
test.to_csv('predictions.csv', index=False)
test.head()

ValueError: Length of values does not match length of index